In [ ]:
# Machine Learning Final
# Connor Hehn and John Minogue

import pandas as pd
import numpy as np

In [ ]:
# Reading the data into data frames

df_1E = pd.read_csv("Data/Subject01E.csv")
df_1G = pd.read_csv("Data/Subject02E.csv")

df_2E = pd.read_csv("Data/Subject02E.csv")
df_2G = pd.read_csv("Data/Subject02G.csv")

df_3E = pd.read_csv("Data/Subject03E.csv")
df_3G = pd.read_csv("Data/Subject03G.csv")

df_4E = pd.read_csv("Data/Subject04E.csv")
df_4G = pd.read_csv("Data/Subject04G.csv")

df_5E = pd.read_csv("Data/Subject05E.csv")
df_5G = pd.read_csv("Data/Subject05G.csv")

df_6E = pd.read_csv("Data/Subject06E.csv")
df_6G = pd.read_csv("Data/Subject06G.csv")

df_7E = pd.read_csv("Data/Subject07E.csv")
df_7G = pd.read_csv("Data/Subject07G.csv")

df_8E = pd.read_csv("Data/Subject08E.csv")
df_8G = pd.read_csv("Data/Subject08G.csv")

df_9E = pd.read_csv("Data/Subject09E.csv")
df_9G = pd.read_csv("Data/Subject09G.csv")

df_10E = pd.read_csv("Data/Subject10E.csv")
df_10G = pd.read_csv("Data/Subject10G.csv")

In [ ]:
# Creating and merging dataframe

frame = [df_1E, df_1G, df_2E, df_2G, df_3E, df_3G, df_4E, df_4G, df_5E, df_5G, df_6E, df_6G, df_7E, df_7G, df_8E, df_8G, df_9E, df_9G, df_10E, df_10G]
maindf = pd.concat(frame)

print(maindf)
